<a href="https://colab.research.google.com/github/FREDY129053/Final_theory_work/blob/main/Analitica_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import pandas as pd

df = pd.read_csv('/content/answers.csv')
df.head()

,Unnamed: 0,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,0,фарпост работа владивосток,NaN,NaN,NaN,NaN,общая фраза
1,1,кофейни вакансии,NaN,NaN,NaN,NaN,общая фраза
2,2,работа разнорабочие часовой,NaN,Рабочий,NaN,NaN,NaN
3,3,личный водитель на день,на неполный день,Водитель,NaN,NaN,NaN
4,4,работа от работодателя персональный водитель,NaN,Водитель,NaN,NaN,NaN


In [36]:
copy_df = df.copy()

In [37]:
import json

with open('/content/parse_data.json', 'r', encoding='utf-8') as file:
  parsed_data = json.load(file)

professions = parsed_data[0]['items']
additional = parsed_data[1]['items']
conditions = parsed_data[2]['items']
employment = parsed_data[3]['items']

In [38]:
# Данные из парсинга для соединения с существующим массивом
data = []

In [39]:
def add_rows(category_data, column):
  for item in category_data:
    row = {
        'query': item,
        'занятость': None,
        'по должности-лемме': None,
        'по дополнительному признаку': None,
        'по условиям': None,
        'общие фразы': None,
    }
    row[column] = item
    data.append(row)

In [40]:
import pandas as pd

add_rows(professions, 'по должности-лемме')
add_rows(additional, 'по дополнительному признаку')
add_rows(conditions, 'по условиям')
add_rows(employment, 'занятость')

parse_df = pd.DataFrame(data)
parse_df.head()

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,HR бизнес-партнер,None,HR бизнес-партнер,None,None,None
1,Аварийный комиссар,None,Аварийный комиссар,None,None,None
2,Авиамеханик,None,Авиамеханик,None,None,None
3,Автодиагност,None,Автодиагност,None,None,None
4,Автомаляр,None,Автомаляр,None,None,None


In [41]:
print(copy_df.shape)
print(parse_df.shape)

(14245, 7)
(562, 6)


In [42]:
combined_df = pd.concat([copy_df, parse_df], ignore_index=True)
combined_df.shape

(14807, 7)

In [43]:
test_df_main_words = combined_df.copy()

In [44]:
copy_df = combined_df.copy()

copy_df['занятость'] = copy_df['занятость'].apply(lambda x: 0 if pd.isna(x) or x is None else 1)
copy_df['по должности-лемме'] = copy_df['по должности-лемме'].apply(lambda x: 0 if pd.isna(x) or x is None else 1)
copy_df['по дополнительному признаку'] = copy_df['по дополнительному признаку'].apply(lambda x: 0 if pd.isna(x) or x is None else 1)
copy_df['по условиям'] = copy_df['по условиям'].apply(lambda x: 0 if pd.isna(x) or x is None else 1)
copy_df['общие фразы'] = copy_df['общие фразы'].apply(lambda x: 0 if pd.isna(x) or x is None else 1)

copy_df = copy_df[['query', 'занятость', 'по должности-лемме', 'по дополнительному признаку', 'по условиям', 'общие фразы']]
copy_df.head()
# 00001 00001 01000 11000 01000

,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,фарпост работа владивосток,0,0,0,0,1
1,кофейни вакансии,0,0,0,0,1
2,работа разнорабочие часовой,0,1,0,0,0
3,личный водитель на день,1,1,0,0,0
4,работа от работодателя персональный водитель,0,1,0,0,0


In [45]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_predict
from keras.models import load_model
import pickle
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

copy_df['query'] = copy_df['query'].apply(preprocess_text)

# Разделение данных на обучающую и тестовую выборки
# X_train, X_test, y_train, y_test = train_test_split(
#     copy_df['query'],
#     copy_df.drop(columns=['query']),
#     test_size=0.2,
#     random_state=42)

# Преобразование текста в векторное представление и обучение модели в одном пайплайне
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', OneVsRestClassifier(LogisticRegression(C=10)))
])

cv = KFold(n_splits=10, shuffle=True, random_state=42)
# y_strat = copy_df.drop(columns=['query']).apply(lambda row: '_'.join(row.astype(str)), axis=1)
y_pred = cross_val_predict(pipeline, copy_df['query'], copy_df.drop(columns=['query']), cv=cv, method='predict')
print(classification_report(copy_df.drop(columns=['query']), y_pred, zero_division=0))
pipeline.fit(copy_df['query'], copy_df.drop(columns=['query']))

# pipeline.fit(X_train, y_train)

# # Предсказание классов
# y_pred = pipeline.predict(X_test)

# # Оценка модели
# print(classification_report(y_test, y_pred))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown i

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1562
           1       0.99      0.93      0.96      8426
           2       1.00      0.97      0.98       855
           3       1.00      0.98      0.99       568
           4       0.92      0.98      0.95      5223

   micro avg       0.97      0.95      0.96     16634
   macro avg       0.98      0.96      0.97     16634
weighted avg       0.97      0.95      0.96     16634
 samples avg       0.95      0.95      0.95     16634



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=10)))])

In [46]:
import joblib

joblib.dump(pipeline, '/content/drive/MyDrive/classification_model.pkl')

['/content/drive/MyDrive/classification_model.pkl']

In [ ]:
def hyperparameter_tuning(X_train, y_train):
    parameters = {
        'tfidf__max_features': [5000, 10000, 15000],
        'clf__estimator__C': [0.1, 1, 10]
    }

    grid_search = GridSearchCV(pipeline, parameters, cv=3, scoring='f1_macro')
    grid_search.fit(X_train, y_train)

    print("Лучшие параметры: ", grid_search.best_params_)
    print("Лучший F1-score: ", grid_search.best_score_)

    return grid_search.best_estimator_

# Подбор гиперпараметров
best_model = hyperparameter_tuning(X_train, y_train)

# Оценка лучшей модели
y_pred_best = best_model.predict(X_test)
print(classification_report(y_test, y_pred_best))

In [47]:
import numpy as np

def classify_query(query):
    query = preprocess_text(query)
    columns = ["Занятость", "По должности", "По дополнительному признаку", "По условиям", "Общая фраза"]
    predict_indexes = pipeline.predict([query])[0]
    return predict_indexes

In [48]:
# classify_query("вакансии мастер")
print(["Занятость", "По должности", "По дополнительному признаку", "По условиям", "Общая фраза"])
print(classify_query("1xbet"))

['Занятость', 'По должности', 'По дополнительному признаку', 'По условиям', 'Общая фраза']
[0 0 0 0 1]


## Telegram-Бот

In [13]:
!pip install aiogram

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.6/570.6 kB 3.7 MB/s eta 0:00:00


In [22]:
import asyncio
import logging
import numpy as np

from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram import F


bot = Bot(token='7110563723:AAEsj1Nrtr3s1wX-WsTgrvu-RFQ6TTGt75E')
dp = Dispatcher()


@dp.message(Command('start'))
async def start_command(message: types.Message):
    await message.answer('Введите текст, а я его классифицирую по данным с FarPost')

@dp.message(Command('help'))
async def help_command(message: types.Message):
    await message.answer('Бот предназначен для классификации запросов\nОбучение происходило на основе датасета компании FarPost')

@dp.message(F.text)
async def get_text_to_predict(message: types.Message):
  user_text = message.text
  await message.answer(f'{get_prediction(user_text)}')



def get_prediction(text):
    query = preprocess_text(text)
    columns = ["Занятость", "По должности", "По дополнительному признаку", "По условиям", "Общая фраза"]
    predict_indexes = pipeline.predict([query])[0]
    if sum(predict_indexes) == 0:
      return 'Общая фраза'

    result = ''
    for i in range(len(predict_indexes)):
      if i == 0 and predict_indexes[i] == 1:
        result += f'{columns[i]}'
      elif predict_indexes[i] == 1:
        result += f' {columns[i]} '
    return f'{result}'


async def main():
    await dp.start_polling(bot)



await main()


ERROR:aiogram.dispatcher:Failed to fetch updates - TelegramNetworkError: HTTP Client says - ClientOSError: [Errno 104] Connection reset by peer


## GPT Model

In [49]:
test_df_main_words.isna()

,Unnamed: 0,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,False,False,True,True,True,True,False
1,False,False,True,True,True,True,False
2,False,False,True,False,True,True,True
3,False,False,False,False,True,True,True
4,False,False,True,False,True,True,True
...,...,...,...,...,...,...,...
14802,True,False,False,True,True,True,True
14803,True,False,False,True,True,True,True
14804,True,False,False,True,True,True,True
14805,True,False,False,True,True,True,True


In [50]:
test_df_main_words.tail(3)

,Unnamed: 0,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
14804,NaN,Временная работа,Временная работа,None,None,None,None
14805,NaN,Работа по выходным,Работа по выходным,None,None,None,None
14806,NaN,Работа на неполный день,Работа на неполный день,None,None,None,None


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
import nltk

# Загрузка необходимых ресурсов NLTK
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление специальных символов
    text = re.sub(r'\W', ' ', text)
    # Токенизация
    words = word_tokenize(text)
    # Удаление стоп-слов
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)


# Загрузка данных
df = test_df_main_words.copy()

# Заполнение пропусков пустыми строками
df.fillna('', inplace=True)

# Объединение всех классов в один столбец
df['labels'] = df.apply(lambda row: [row['занятость'], row['по должности-лемме'], row['по дополнительному признаку'], row['по условиям'], row['общие фразы']], axis=1)

# Очистка строковых данных и их токенизация
stop_words = set(stopwords.words('russian'))

df['clean_query'] = df['query'].apply(preprocess_text)

# Разделение данных на признаки и метки
X = df['clean_query']
y = df['labels']

# Векторизация текстовых данных
vectorizer = TfidfVectorizer(max_features=5000)

X_vectorized = vectorizer.fit_transform(X)

# Преобразование меток в формат, удобный для MultiOutputClassifier

mlb = MultiLabelBinarizer()

y_binarized = mlb.fit_transform(y)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_binarized, test_size=0.2, random_state=42)

# Определение модели
rf = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)

# Обучение модели
multi_target_rf.fit(X_train, y_train)

# Оценка модели
y_pred = multi_target_rf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


In [ ]:
# Пример использования модели
def classify_query(query):
    # Предобработка запроса
    clean_query = preprocess_text(query)
    # Векторизация
    query_vectorized = vectorizer.transform([clean_query])
    # Предсказание
    prediction = multi_target_rf.predict(query_vectorized)
    # Преобразование предсказаний в метки
    predicted_labels = mlb.inverse_transform(prediction)
    return predicted_labels

query = "работа дизайнером дневная"
predicted_labels = classify_query(query)
print(f"Запрос: '{query}'\nКлассы: {predicted_labels}")

In [ ]:
import joblib

joblib.dump(multi_target_rf, '/content/drive/MyDrive/main_words_recognition.pkl')

In [27]:
temp_model = joblib.load('/content/main_words_recognition.pkl')

# Предобработка запроса
clean_query = preprocess_text('работа дизайнером дневная')
# Векторизация
query_vectorized = vectorizer.transform([clean_query])
# Предсказание
prediction = temp_model.predict(query_vectorized)
# Преобразование предсказаний в метки
predicted_labels = mlb.inverse_transform(prediction)
print(predicted_labels)

# print(temp_model.predict(('работа дизайнером дневная')))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


[('', 'Дизайнер', 'Дневная')]
